In [130]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
import time
import pandas as pd
from numpy import e

In [142]:
def job_posts(page_number=2,profession='Data Scientist',location='United States'):       # Define a function that takes page number, profession and location as arguments.
    if ' ' in profession:                                                                # Optimize the link for LinkedIn. 
        profession = profession.replace(' ','%20')                     
    if ' ' in location:                                                                  # Optimize the link for LinkedIn. 
        location = location.replace(' ','%20')
    driver = webdriver.Chrome()                                                          # Instantiate the Chrome browser
    links = []                                                                           # Create an empty links list to use later as storage.
    position_names = []                                                                  # Create an empty position names list to use later as storage.
    companies = []                                                                       # Create an empty companies list to use later as storage.
    locations = []                                                                       # Create an empty locations list to use later as storage.
    path1 = f'https://www.linkedin.com/jobs/search?keywords={profession}&location={location}'  # The standard link sequence which can be modified by the user inputs 
    driver.get(path1)                                                                    # Browser navigates to the link
    i = 0                                                                                # Base number that helps with arranging the iteration number
    while i < page_number:                                                               # While loop that helps with scrolling. 
        html = driver.find_element(By.TAG_NAME,'html')                                   # Select anything on the page
        html.send_keys(Keys.END)                                                         # Send 'END' key to scroll down the page 
        i = i+1                                                                          # i increases incrementally. This helps with the number of repeats
        try:                                                                             # If the HTML does not include the desired element, the flow is protected with try func.
            t = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/button')# Find 'See more Jobs' button
            t.click()                                                                    # Click the button
            time.sleep(2)                                                                # Let the page load for 2 seconds.
        except:                                                                          # If a problem arises the flow continues. 
            pass
            time.sleep(3)                                                                # Wait for 3 secs
    block = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/ul')        # Find 
    job_list = block.find_elements(By.CLASS_NAME, 'base-card')
    try:
        for job in job_list:
            try:
                link = job.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href')
                links.append(link)
            except:
                links.append(None)
                ValueError('no link')
            try:
                position_name = job.find_element(By.TAG_NAME,'span').text
                position_names.append(position_name)
            except:
                position_names.append(None)
                ValueError('no pos name')
            try:
                company = job.find_element(By.CLASS_NAME,'hidden-nested-link').text
                companies.append(company)
            except:
                companies.append(None)
                ValueError('no company')
            try:
                location = job.find_element(By.CLASS_NAME,'job-search-card__location').text
                locations.append(location)
            except:
                locations.append(None)
                ValueError('no location')
    except:
        e = 'no path'
        ValueError(e)
        print(e)
    data = pd.DataFrame({'Position':position_names,
            'Company':companies,
            'Location':locations,
            'Link':links})     
    return data

In [152]:
df=job_posts(2,profession='Analyst',location='United States')

In [136]:
def get_content(P_no=2,prof='Data Scientist',loc='United States'):
    df = job_posts(page_number = P_no,profession=prof,location=loc)
    links=df.Link.tolist()
    driver = webdriver.Chrome()
    levels = []
    linkss =[] 
    employment = []
    job_functions = []
    industries = []
    contents = []
    for link in links:
        if len(contents)%25 == 0 and len(contents)>0:
            print(f'Scraping job number:{len(contents)}...')    
        try:
            driver.get(link)
            driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
            time.sleep(1)     
            try:
                linkss.append(link)
            except:
                linkss.append(None)
                ValueError('No link')
            try:
                Seniority_Level = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[1]/span').text
                levels.append(Seniority_Level)
            except:
                levels.append(None)
                ValueError('No level')
            try:
                Employment_Type = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[2]/span').text
                employment.append(Employment_Type)
            except:
                employment.append(None)
                ValueError('no employment')  
            try:
                Job_Function = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[3]/span').text
                job_functions.append(Job_Function)
            except:
                job_functions.append(None)
                ValueError('no job function')
            try:
                Industry = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[4]/span').text
                industries.append(Industry)
            except:
                industries.append(None)
                ValueError('no industry')     
            try:
                content=driver.find_element(By.CLASS_NAME,'show-more-less-html__markup').text
                contents.append(content)
            except:
                contents.append(None)
                ValueError('no content')     
        except:
            ValueError('smthng is wrong')
        time.sleep(5)
    c_data = pd.DataFrame({'links':linkss,
                            'content':contents,
                            'level':levels,
                            'job_function':job_functions,
                            'industries':industries,
                            'employment':employment})
    df_merged = c_data.merge(df,how='left',right_on='Link',left_on='links')
    df_merged = df_merged.drop(columns=['Link','links'])
    return df_merged

In [153]:
df1=get_content(P_no=2,prof='Data Scientist',loc='United States')

Scraping job number:25...
Scraping job number:25...


,content,level,job_function,industries,employment,Position,Company,Location
0,Genesis is building a world-class ML team in a...,Not Applicable,Engineering and Information Technology,"Transportation, Logistics, Supply Chain and St...",Full-time,ML Research Engineer (New Grad),Genesis Therapeutics,"New York, NY"
1,Genesis is building a world-class ML team in a...,Not Applicable,Engineering and Information Technology,"Transportation, Logistics, Supply Chain and St...",Full-time,ML Research Engineer (New Grad),Genesis Therapeutics,"Burlingame, CA"
2,Jaan Health’s team is growing! We’re looking f...,Mid-Senior level,"Engineering, Information Technology, and Research","Hospitals and Health Care, Software Developmen...",Full-time,Machine Learning Engineer,Phamily,United States
3,Senior Manger WFS Strategy & Data Science\nWal...,Associate,"Analyst, Design, and Supply Chain","Retail Apparel and Fashion, Transportation, Lo...",Full-time,Senior Data Scientist,Walmart,"Seattle, WA"
4,Type: Full Time\n\nClearance Required: Top Sec...,Mid-Senior level,Engineering and Information Technology,Internet Publishing,Full-time,Data Scientist,"Noetic Strategies, Inc.","Scott AFB, IL"
5,Dun & Bradstreet unlocks the power of data thr...,Associate,"Engineering, Information Technology, and Science","IT Services and IT Consulting, Information Ser...",Full-time,Junior Data Scientist,Dun & Bradstreet,"Florham Park, NJ"
6,Position: Data Scientist\n\nLocation: Santa Cl...,Mid-Senior level,Engineering and Information Technology,"Technology, Information and Internet",Full-time,DATA SCIENTIST,PETADATA,"Santa Clarita, CA"
7,"Founded in 2003, IT Concepts' core values - cu...",Mid-Senior level,Analyst,"Technology, Information and Internet",Full-time,Data Scientist (Healthcare),"IT Concepts, Inc","District of Columbia, United States"
8,This role is responsible for analysis & modeli...,Associate,Analyst and Information Technology,"Transportation, Logistics, Supply Chain and St...",Full-time,Junior Data Scientist,BlueGrace Logistics,"Riverview, FL"
9,Role:- Sr. Data Scientist\n\nRemote\n\nRespons...,Mid-Senior level,Engineering and Information Technology,IT Services and IT Consulting,Full-time,Data Scientist,PSRTEK,"New York, NY"


In [158]:
df1.to_csv('/Users/ritvik/Documents/GitHub/linkedinscraper/LinkedIn_jobs.csv')